<a href="https://colab.research.google.com/github/ajtamayoh/ClinicalTextMining/blob/main/Negation_and_Uncertainty_Detection_in_Spanish_with_the_NUBES_dataset_shared_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clinical Text Mining in Spanish (Negation and Uncertainty Detection)

Here you are the source code for the paper:

### Clinical Text Mining in Spanish Enhanced by Negation Detection and Named Entity Recognition

Authors:

Antonio Tamayo (ajtamayo2019@ipn.cic.mx, ajtamayoh@gmail.com)

Diego A. Burgos (burgosda@wfu.edu)

Alexander Gelbulkh (gelbukh@gelbukh.com)

For bugs or questions related to the code, do not hesitate to contact us (Antonio Tamayo: ajtamayoh@gmail.com)

If you use this code please cite our work:

Comming soon...



# Requirements

To run this code you need to download the dataset (three files: ) at: [download dataset]()

Then, you must create a folder called "Datasets" in the root of your Google Drive and load there both folders previously downloaded.

Once the dataset is ready to use, you should [open this notebook in colab]() and save a copy in your drive.

## About the infrastructure

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Aug 26 03:09:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    34W / 250W |    407MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


## Install the Transformers and Datasets libraries to run this notebook.

In [ ]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 13.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=240534c062b391616a373e6107d0a9f4f392c9172e7d15da82e1c1f08c4df357
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


## Hugging Face Authentication

If you want to save your own model and make it available online we strongly recommend signing up at: https://huggingface.co/

You will need to setup git, adapt your email and name in the following cell.

In [ ]:
!git config --global user.email "ajtamayoh@gmail.com"
!git config --global user.name "ajtamayoh"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Connecting to Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Exploring & Preprocessing Data

In [ ]:
import pandas as pd
import numpy as np
import spacy

# Negation Scope Detection as a token classification problem

In [ ]:
from datasets import load_dataset

#Loading the dataset
NubEs_tr = load_dataset("json", data_files="/content/drive/MyDrive/Datasets/JSON_NubEs_train_HF.json", field="data")
NubEs_val = load_dataset("json", data_files="/content/drive/MyDrive/Datasets/JSON_NubEs_dev_HF.json", field="data")
NubEs_test = load_dataset("json", data_files="/content/drive/MyDrive/Datasets/JSON_NubEs_test_HF.json", field="data")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e238d9d05ea340f5/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-b64c2769d6a96ef8/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

'\n#Aquí cargo mi versión de Nubes solo para NSD\nNubEs_tr = load_dataset("json", data_files="/content/drive/MyDrive/Doctorado CIC-IPN/NegNER/NubEs Corpus/NubEs_Only_Negations_Original/JSON_NubEs_train_Only_NEG_HF.json", field="data")\nNubEs_val = load_dataset("json", data_files="/content/drive/MyDrive/Doctorado CIC-IPN/NegNER/NubEs Corpus/NubEs_Only_Negations_Original/JSON_NubEs_dev_Only_NEG_HF.json", field="data")\nNubEs_test = load_dataset("json", data_files="/content/drive/MyDrive/Doctorado CIC-IPN/NegNER/NubEs Corpus/NubEs_Only_Negations_Original/JSON_NubEs_test_Only_NEG_HF.json", field="data")\n'

In [ ]:
NubEs_tr.shape
NubEs_val.shape
NubEs_test.shape

{'train': (2761, 2)}

In [ ]:
NubEs_tr

DatasetDict({
    train: Dataset({
        features: ['ner_tags', 'tokens'],
        num_rows: 13801
    })
})

## Building the Dataset

In [ ]:
from datasets import DatasetDict
#For training, validation, and test partitions

raw_datasets = DatasetDict({
    'train': NubEs_tr['train'],
    'validation': NubEs_val['train'],
    'test': NubEs_test['train']
    })


In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13801
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1839
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2761
    })
})

In [ ]:
raw_datasets["train"][0]["ner_tags"]

[0, 0, 0, 0, 0, 0]

In [ ]:
raw_datasets['train']

Dataset({
    features: ['ner_tags', 'tokens'],
    num_rows: 13801
})

In [ ]:
#Para NSD y Uncertainty Scope Detection (USD)
label_names = [
                                "O",        # 0
                                "B-UNC",    # 1
                                "B-NSCO",   # 2
                                "B-USCO",   # 3
                                "B-NEG",    # 4
                                "I-UNC",    # 5
                                "I-NSCO",   # 6
                                "I-USCO",   # 7
                                "I-NEG",    # 8
                            ]
label_names

['O',
 'B-UNC',
 'B-NSCO',
 'B-USCO',
 'B-NEG',
 'I-UNC',
 'I-NSCO',
 'I-USCO',
 'I-NEG']

In [ ]:
words = raw_datasets["train"][0]["tokens"]
labels = [int(n) for n in raw_datasets["train"][0]["ner_tags"]]
#labels = raw_datasets["train"][0]["pos_tags"]
#labels = raw_datasets["train"][0]["chunk_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

OD : NVC que precisó TFD 
O  O O   O   O       O   


## Loading BERT-based model as a pre-trained one

In [ ]:
from transformers import AutoTokenizer

#model_checkpoint = "PlanTL-GOB-ES/roberta-base-biomedical-clinical-es"
model_checkpoint = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

In [ ]:
tokenizer.is_fast

True

In [ ]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'O',
 '##D',
 ':',
 'N',
 '##V',
 '##C',
 'que',
 'pre',
 '##cis',
 '##ó',
 'T',
 '##F',
 '##D',
 '[SEP]']

In [ ]:
inputs.word_ids()

[None, 0, 0, 1, 2, 2, 2, 3, 4, 4, 4, 5, 5, 5, None]

In [ ]:
tokenizer(["El paciente hipertenso no presenta fiebre ni infección."]).tokens()

['[CLS]',
 'El',
 'paciente',
 'hip',
 '##erten',
 '##so',
 'no',
 'presenta',
 'fie',
 '##bre',
 'ni',
 'in',
 '##fección',
 '.',
 '[SEP]']

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/13801 [00:00<?, ? examples/s]

Map:   0%|          | 0/1839 [00:00<?, ? examples/s]

Map:   0%|          | 0/2761 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,    2,
            2,    5,    3,    4,    4,    7,    8,    8,    8,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0, -100]])

In [ ]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 5, 3, 4, 4, 7, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [ ]:
from datasets import load_metric

metric = load_metric("seqeval")

<ipython-input-23-edef154daec7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['O', 'O', 'O', 'O', 'O', 'O']

In [ ]:
predictions = labels.copy()
predictions[2] = "B-NSCO"
metric.compute(predictions=[predictions], references=[labels])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'NSCO': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.8333333333333334}

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        
        #per label

        #Negation
        "NEG_precision": all_metrics["NEG"]['precision'],
        "NEG_recall": all_metrics["NEG"]['recall'],
        "NEG_F1": all_metrics["NEG"]['f1'],
        "NSCO_precision": all_metrics["NSCO"]['precision'],
        "NSCO_recall": all_metrics["NSCO"]['recall'],
        "NSCO_F1": all_metrics["NSCO"]['f1'],
        #Uncertainty
        "UNC_precision": all_metrics["UNC"]['precision'],
        "UNC_recall": all_metrics["UNC"]['recall'],
        "UNC_F1": all_metrics["UNC"]['f1'],
        "USCO_precision": all_metrics["USCO"]['precision'],
        "USCO_recall": all_metrics["USCO"]['recall'],
        "USCO_F1": all_metrics["USCO"]['f1'],

        #Overall
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
id2label

{'0': 'O',
 '1': 'B-UNC',
 '2': 'B-NSCO',
 '3': 'B-USCO',
 '4': 'B-NEG',
 '5': 'I-UNC',
 '6': 'I-NSCO',
 '7': 'I-USCO',
 '8': 'I-NEG'}

In [ ]:
label2id

{'O': '0',
 'B-UNC': '1',
 'B-NSCO': '2',
 'B-USCO': '3',
 'B-NEG': '4',
 'I-UNC': '5',
 'I-NSCO': '6',
 'I-USCO': '7',
 'I-NEG': '8'}

## Changing the head of prediction for NER under the IO scheme

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(    
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
    num_labels = 9,    # For join NSD and USD
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [ ]:
model.config.num_labels

9

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
  
    #"Negation_and_Uncertainty_Scope_Detection_RoBERTa_fine_tuned_your_identifier", # Training + Test dataset
    #"Negation_and_Uncertainty_Scope_Detection_RoBERTa_fine_tuned_dev_your_identifier", # Training + Dev dataset
    #"Negation_and_Uncertainty_Scope_Detection_mBERT_fine_tuned_your_identifier", # Training + Test dataset
    "Negation_and_Uncertainty_Scope_Detection_mBERT_fine_tuned_dev_your_identifier", # Training + Dev dataset
    
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=7,
    weight_decay=0.01,
    push_to_hub=True,
)

## Fine-tuning mBERT for Negation Scope Detection

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    #eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Cloning https://huggingface.co/ajtamayoh/Negation_and_Uncertainty_Scope_Detection_mBERT_fine_tuned_dev into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Neg Precision,Neg Recall,Neg F1,Nsco Precision,Nsco Recall,Nsco F1,Unc Precision,Unc Recall,Unc F1,Usco Precision,Usco Recall,Usco F1,Precision,Recall,F1,Accuracy
1,0.025100,0.185202,0.915645,0.925173,0.920385,0.881692,0.921141,0.900985,0.831395,0.922581,0.874618,0.829755,0.834877,0.832308,0.885376,0.911209,0.898107,0.970312
2,0.022300,0.175967,0.916300,0.927651,0.921940,0.880236,0.916667,0.898082,0.861111,0.900000,0.880126,0.817352,0.828704,0.822989,0.885591,0.908270,0.896788,0.967918
3,0.014200,0.186929,0.924089,0.904856,0.914372,0.894508,0.920022,0.907086,0.916107,0.880645,0.898026,0.851485,0.796296,0.822967,0.902734,0.894207,0.898450,0.969883
4,0.010800,0.202700,0.921287,0.922200,0.921743,0.906836,0.920022,0.913381,0.879365,0.893548,0.886400,0.830585,0.854938,0.842586,0.900540,0.910369,0.905428,0.970723
5,0.004000,0.207503,0.914230,0.929633,0.921867,0.898268,0.928412,0.913091,0.857576,0.912903,0.884375,0.844615,0.847222,0.845917,0.895082,0.916877,0.905848,0.970723
6,0.003200,0.209167,0.924779,0.932111,0.928430,0.880952,0.931208,0.905383,0.872340,0.925806,0.898279,0.846861,0.853395,0.850115,0.894007,0.920655,0.907135,0.971437
7,0.002200,0.214409,0.930348,0.926660,0.928500,0.896477,0.925056,0.910542,0.882353,0.919355,0.900474,0.843558,0.848765,0.846154,0.902484,0.914987,0.908693,0.972170


TrainOutput(global_step=12082, training_loss=0.011515135497121182, metrics={'train_runtime': 2243.4207, 'train_samples_per_second': 43.062, 'train_steps_per_second': 5.386, 'total_flos': 3559340712539862.0, 'train_loss': 0.011515135497121182, 'epoch': 7.0})

## Saving the fine-tuned model at Hugging Face (It requires previous authentication)

In [ ]:
trainer.push_to_hub(commit_message="Fine-tuning completed")

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/676M [00:00<?, ?B/s]

Upload file runs/May29_16-05-49_7f41949ffa70/events.out.tfevents.1685376365.7f41949ffa70.1352.2:   0%|        …

To https://huggingface.co/ajtamayoh/Negation_and_Uncertainty_Scope_Detection_mBERT_fine_tuned_dev
   dd486e0..9c13c25  main -> main

   dd486e0..9c13c25  main -> main

To https://huggingface.co/ajtamayoh/Negation_and_Uncertainty_Scope_Detection_mBERT_fine_tuned_dev
   9c13c25..d7dff38  main -> main

   9c13c25..d7dff38  main -> main



'https://huggingface.co/ajtamayoh/Negation_and_Uncertainty_Scope_Detection_mBERT_fine_tuned_dev/commit/9c13c25423f9f286226bc031279ab3dd22bdc5f2'

## Loading the model for inference

In [ ]:
from transformers import pipeline

#Replace this with your own checkpoint.

model_checkpoint = "user_name/model_identifier"

token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="simple")

https://huggingface.co/ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpqb3vza2u


storing https://huggingface.co/ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/74da0d12fe75d57d89c78b6f544782d7aa86fdc62f943e7ea49706349f8d9300.c144e91635dd448138d13e58f93fb9ec74a1311ae70917824e4f4416e7a260f7
creating metadata file for /root/.cache/huggingface/transformers/74da0d12fe75d57d89c78b6f544782d7aa86fdc62f943e7ea49706349f8d9300.c144e91635dd448138d13e58f93fb9ec74a1311ae70917824e4f4416e7a260f7
loading configuration file https://huggingface.co/ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/74da0d12fe75d57d89c78b6f544782d7aa86fdc62f943e7ea49706349f8d9300.c144e91635dd448138d13e58f93fb9ec74a1311ae70917824e4f4416e7a260f7
Model config BertConfig {
  "_name_or_path": "ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned",
  "architectures": [
    "BertForTokenClassification"
  ],
  "atte

storing https://huggingface.co/ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/599e564848b4e3dbc7ccfa22b9bfa1dd4dfc2e1c2c48d4052d096be09a05a6f1.5f62145734a371d759b089b6391b446f6628b7b3c5eec2ad4bba0fdb28ab629a
creating metadata file for /root/.cache/huggingface/transformers/599e564848b4e3dbc7ccfa22b9bfa1dd4dfc2e1c2c48d4052d096be09a05a6f1.5f62145734a371d759b089b6391b446f6628b7b3c5eec2ad4bba0fdb28ab629a
loading weights file https://huggingface.co/ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/599e564848b4e3dbc7ccfa22b9bfa1dd4dfc2e1c2c48d4052d096be09a05a6f1.5f62145734a371d759b089b6391b446f6628b7b3c5eec2ad4bba0fdb28ab629a
All model checkpoint weights were used when initializing BertForTokenClassification.

All the weights of BertForTokenClassification were initialized from the model checkpoint

storing https://huggingface.co/ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/5cefc0ed90c0a6d91691dcd7586b2e4c08b72222cc21c4d1649b487251023bd5.a363461fb3b0f1af29dfb91fd4431cbf8a76c7ea40502936c51d38477a272f27
creating metadata file for /root/.cache/huggingface/transformers/5cefc0ed90c0a6d91691dcd7586b2e4c08b72222cc21c4d1649b487251023bd5.a363461fb3b0f1af29dfb91fd4431cbf8a76c7ea40502936c51d38477a272f27
https://huggingface.co/ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpqwmfpx2h


storing https://huggingface.co/ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/ad8cd824e2156f6797268c619cd9e01ccee5ed45c3afd23b5063980afb9b2734.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
creating metadata file for /root/.cache/huggingface/transformers/ad8cd824e2156f6797268c619cd9e01ccee5ed45c3afd23b5063980afb9b2734.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
https://huggingface.co/ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpd2cxt9iz


storing https://huggingface.co/ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/73fa311feb3b83e7858389119c544a1100523f9b1253a97f2b0592c65cb75125.71c89bf3b4c92a8eea726f80cab885a5157912e692f722e0a31b8275c7c6a1a6
creating metadata file for /root/.cache/huggingface/transformers/73fa311feb3b83e7858389119c544a1100523f9b1253a97f2b0592c65cb75125.71c89bf3b4c92a8eea726f80cab885a5157912e692f722e0a31b8275c7c6a1a6
https://huggingface.co/ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpjur9ud83


storing https://huggingface.co/ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/be265c205e0b560a248b32488b5ee22d66e6c726f9b3537b890ccedd78653f6c.7da70648c6cb9951e284c9685f9ba7ae083dd59ed1d6d84bdfc0584a4ea94b6d
creating metadata file for /root/.cache/huggingface/transformers/be265c205e0b560a248b32488b5ee22d66e6c726f9b3537b890ccedd78653f6c.7da70648c6cb9951e284c9685f9ba7ae083dd59ed1d6d84bdfc0584a4ea94b6d
loading file https://huggingface.co/ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/ad8cd824e2156f6797268c619cd9e01ccee5ed45c3afd23b5063980afb9b2734.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/ajtamayoh/Negation_Scope_Detection_NubEs_Spanish_mBERT_fine_tuned/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/73